## 1. Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

# SHAP for interpretability
import shap

print("Libraries imported successfully!")
print(f"SHAP version: {shap.__version__}")

## 2. Load Best Model and Data

In [ ]:
# Load model comparison results to identify best model
results_df = pd.read_csv('../models/model_comparison_results.csv')
best_model_name = results_df.loc[results_df['Test RMSE'].idxmin(), 'Model']

print(f"Best Model: {best_model_name}")
print(f"Test RMSE: {results_df['Test RMSE'].min():.4f}")
print(f"Test R²: {results_df.loc[results_df['Test RMSE'].idxmin(), 'Test R2']:.4f}")

In [ ]:
# Determine which dataset the best model uses
linear_models = ['Ridge Regression', 'Lasso Regression', 'ElasticNet']
catboost_models = ['CatBoost']

if best_model_name in linear_models:
    dataset_path = '../data/data_processed/dataset_linear_models.pkl'
    model_file = best_model_name.lower().replace(' ', '_')
elif best_model_name in catboost_models:
    dataset_path = '../data/data_processed/dataset_catboost.pkl'
    model_file = 'catboost'
else:
    dataset_path = '../data/data_processed/dataset_tree_models.pkl'
    model_file = best_model_name.lower().replace(' ', '_')

print(f"\nLoading dataset: {dataset_path}")
print(f"Loading model: {model_file}_best.pkl")

In [ ]:
# Load the dataset
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)
    X_train = data['X_train']
    X_test = data['X_test']
    y_train = data['y_train']
    y_test = data['y_test']

print(f"\nData loaded:")
print(f"  Training set: {X_train.shape}")
print(f"  Test set: {X_test.shape}")
print(f"  Features: {X_train.shape[1]}")

In [ ]:
# Load the best model
model_path = f'../models/{model_file}_best.pkl'
with open(model_path, 'rb') as f:
    model = pickle.load(f)

print(f"Model loaded: {model_path}")
print(f"Model type: {type(model).__name__}")

## 3. Initialize SHAP Explainer

In [ ]:
# Initialize SHAP with JavaScript visualization
shap.initjs()

print("Creating SHAP explainer...")
print("This may take a few minutes...\n")

# Use a sample of training data for efficiency (500 samples)
X_train_sample = shap.sample(X_train, 500, random_state=42)

# Create appropriate explainer based on model type
model_type = type(model).__name__

if 'XGB' in model_type or 'LGB' in model_type or 'CatBoost' in model_type:
    # Tree-based models use TreeExplainer (fast and exact)
    explainer = shap.TreeExplainer(model)
    print(f"Using TreeExplainer for {model_type}")
else:
    # Linear models use LinearExplainer (fast and exact)
    # Other models use KernelExplainer (slower but works for any model)
    if hasattr(model, 'coef_'):
        explainer = shap.LinearExplainer(model, X_train_sample)
        print(f"Using LinearExplainer for {model_type}")
    else:
        explainer = shap.KernelExplainer(model.predict, X_train_sample)
        print(f"Using KernelExplainer for {model_type}")

print("✓ Explainer created successfully!")

## 4. Calculate SHAP Values

In [ ]:
print("Calculating SHAP values for test set...")
print("This may take several minutes...\n")

# Calculate SHAP values for test set (use subset for speed)
X_test_sample = X_test.sample(min(1000, len(X_test)), random_state=42)
shap_values = explainer.shap_values(X_test_sample)

print(f"✓ SHAP values calculated!")
print(f"  Shape: {shap_values.shape if hasattr(shap_values, 'shape') else np.array(shap_values).shape}")

## 5. Global Feature Importance

**Summary Plot**: Shows which features are most important overall across all predictions.

In [ ]:
# Summary plot (bar) - Feature importance
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, plot_type="bar", show=False)
plt.title('Feature Importance (Mean |SHAP Value|)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Features at the top have the strongest impact on RiskScore predictions")
print("- The bar length shows average absolute impact across all predictions")

In [ ]:
# Summary plot (beeswarm) - Feature importance with impact direction
plt.figure(figsize=(10, 12))
shap.summary_plot(shap_values, X_test_sample, show=False)
plt.title('Feature Impact on RiskScore', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Each dot is one prediction")
print("- Red = high feature value, Blue = low feature value")
print("- Right side (positive SHAP) = increases RiskScore (more risky)")
print("- Left side (negative SHAP) = decreases RiskScore (less risky)")

## 6. Individual Prediction Explanations

**Waterfall plots**: Show how each feature contributes to a specific prediction.

In [ ]:
# Get predictions for the test sample
y_test_sample = y_test.loc[X_test_sample.index]
predictions = model.predict(X_test_sample)

# Identify interesting cases
low_risk_idx = predictions.argmin()  # Lowest predicted risk
high_risk_idx = predictions.argmax()  # Highest predicted risk
median_risk_idx = np.argsort(predictions)[len(predictions)//2]  # Median risk

print(f"Selected examples for detailed analysis:")
print(f"  Low Risk:    Predicted={predictions[low_risk_idx]:.2f}, Actual={y_test_sample.iloc[low_risk_idx]:.2f}")
print(f"  Median Risk: Predicted={predictions[median_risk_idx]:.2f}, Actual={y_test_sample.iloc[median_risk_idx]:.2f}")
print(f"  High Risk:   Predicted={predictions[high_risk_idx]:.2f}, Actual={y_test_sample.iloc[high_risk_idx]:.2f}")

In [ ]:
# Waterfall plot for LOW RISK applicant
print("\n" + "="*80)
print("EXPLANATION FOR LOW RISK APPLICANT")
print("="*80)

plt.figure(figsize=(10, 8))
shap.waterfall_plot(shap.Explanation(values=shap_values[low_risk_idx], 
                                     base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else predictions.mean(),
                                     data=X_test_sample.iloc[low_risk_idx],
                                     feature_names=X_test_sample.columns.tolist()), 
                    show=False)
plt.title(f'Low Risk Prediction Explanation (RiskScore = {predictions[low_risk_idx]:.2f})', 
          fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- E[f(x)] = Expected RiskScore (average across all applicants)")
print("- Each bar shows how a feature pushes the prediction up (red) or down (blue)")
print("- f(x) = Final predicted RiskScore for this specific applicant")

In [ ]:
# Waterfall plot for HIGH RISK applicant
print("\n" + "="*80)
print("EXPLANATION FOR HIGH RISK APPLICANT")
print("="*80)

plt.figure(figsize=(10, 8))
shap.waterfall_plot(shap.Explanation(values=shap_values[high_risk_idx], 
                                     base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else predictions.mean(),
                                     data=X_test_sample.iloc[high_risk_idx],
                                     feature_names=X_test_sample.columns.tolist()), 
                    show=False)
plt.title(f'High Risk Prediction Explanation (RiskScore = {predictions[high_risk_idx]:.2f})', 
          fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Waterfall plot for MEDIAN RISK applicant
print("\n" + "="*80)
print("EXPLANATION FOR MEDIAN RISK APPLICANT")
print("="*80)

plt.figure(figsize=(10, 8))
shap.waterfall_plot(shap.Explanation(values=shap_values[median_risk_idx], 
                                     base_values=explainer.expected_value if hasattr(explainer, 'expected_value') else predictions.mean(),
                                     data=X_test_sample.iloc[median_risk_idx],
                                     feature_names=X_test_sample.columns.tolist()), 
                    show=False)
plt.title(f'Median Risk Prediction Explanation (RiskScore = {predictions[median_risk_idx]:.2f})', 
          fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Feature Dependence Plots

Shows how individual features affect RiskScore predictions.

In [ ]:
# Get top 6 most important features
mean_abs_shap = np.abs(shap_values).mean(axis=0)
top_features_idx = np.argsort(mean_abs_shap)[-6:][::-1]
top_features = X_test_sample.columns[top_features_idx].tolist()

print("Top 6 Most Important Features:")
for i, feat in enumerate(top_features, 1):
    print(f"{i}. {feat}")

In [ ]:
# Create dependence plots for top features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, feature in enumerate(top_features):
    plt.sca(axes[idx])
    shap.dependence_plot(feature, shap_values, X_test_sample, ax=axes[idx], show=False)
    axes[idx].set_title(f'{feature}', fontsize=11, fontweight='bold')

plt.suptitle('Feature Dependence Plots - Top 6 Features', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- X-axis: Feature value")
print("- Y-axis: SHAP value (impact on RiskScore)")
print("- Color: Interaction with another feature")
print("- Shows how changing a feature value affects the predicted risk")

## 8. Force Plots - Interactive Explanation

Interactive visualization showing how features push predictions from the base value.

In [ ]:
# Force plot for a single prediction
print("Force plot for HIGH RISK applicant:")
print("(Interactive visualization - red pushes risk higher, blue pushes lower)\n")

shap.force_plot(
    explainer.expected_value if hasattr(explainer, 'expected_value') else predictions.mean(),
    shap_values[high_risk_idx],
    X_test_sample.iloc[high_risk_idx],
    matplotlib=True,
    show=False
)
plt.title(f'Force Plot - High Risk Applicant (RiskScore = {predictions[high_risk_idx]:.2f})', 
          fontsize=12, fontweight='bold', pad=10)
plt.tight_layout()
plt.show()

## 9. Business Insights from SHAP Analysis

In [ ]:
# Calculate feature contribution statistics
feature_importance_df = pd.DataFrame({
    'Feature': X_test_sample.columns,
    'Mean_Abs_SHAP': np.abs(shap_values).mean(axis=0),
    'Mean_SHAP': shap_values.mean(axis=0),
    'Std_SHAP': shap_values.std(axis=0)
}).sort_values('Mean_Abs_SHAP', ascending=False)

print("="*80)
print("KEY INSIGHTS FOR LOAN DECISION MAKERS")
print("="*80)

print("\n1. TOP 10 MOST INFLUENTIAL FEATURES:")
print("="*80)
for idx, row in feature_importance_df.head(10).iterrows():
    direction = "↑ Increases" if row['Mean_SHAP'] > 0 else "↓ Decreases"
    print(f"   {row['Feature']:30s} | Impact: {row['Mean_Abs_SHAP']:.4f} | {direction} risk")

print("\n2. RISK DRIVERS (Features that typically INCREASE RiskScore):")
print("="*80)
risk_drivers = feature_importance_df[feature_importance_df['Mean_SHAP'] > 0].head(5)
for idx, row in risk_drivers.iterrows():
    print(f"   ⚠️  {row['Feature']:30s} | Avg. Impact: +{row['Mean_SHAP']:.4f}")

print("\n3. RISK REDUCERS (Features that typically DECREASE RiskScore):")
print("="*80)
risk_reducers = feature_importance_df[feature_importance_df['Mean_SHAP'] < 0].head(5)
for idx, row in risk_reducers.iterrows():
    print(f"   ✓  {row['Feature']:30s} | Avg. Impact: {row['Mean_SHAP']:.4f}")

print("\n4. RECOMMENDATIONS FOR LOAN OFFICERS:")
print("="*80)
print("   • Pay special attention to the top 3-5 features when reviewing applications")
print("   • For borderline cases, examine risk driver features in detail")
print("   • Request additional documentation for high-impact negative features")
print("   • Use SHAP explanations to justify decisions to customers")
print("   • Monitor if model focuses on fair, non-discriminatory features")
print("="*80)

## 10. Export SHAP Values for Further Analysis

In [ ]:
# Save feature importance ranking
feature_importance_df.to_csv('../models/shap_feature_importance.csv', index=False)
print("✓ Feature importance saved to: ../models/shap_feature_importance.csv")

# Save SHAP values for test set
shap_df = pd.DataFrame(shap_values, columns=X_test_sample.columns, index=X_test_sample.index)
shap_df['Predicted_RiskScore'] = predictions
shap_df['Actual_RiskScore'] = y_test_sample.values
shap_df.to_csv('../models/shap_values_test_set.csv')
print("✓ SHAP values saved to: ../models/shap_values_test_set.csv")

print("\n" + "="*80)
print("SHAP ANALYSIS COMPLETE!")
print("="*80)
print("\nYou now have:")
print("  1. Global feature importance rankings")
print("  2. Individual prediction explanations")
print("  3. Feature dependence patterns")
print("  4. Business insights for decision makers")
print("  5. Exported data for further analysis")